In [50]:
import os
from typing import List, Optional, Tuple


# Parameters
GROUP_ID: str = 'Group29'
ALGORITHM: str = 'bt'
PUZZLE_TYPE: str = 'easy'
PUZZLE_PATH: str = 'puzzles/escargot.txt'

EMPTY_VALUE: int = 0  # Note this value is somewhat arbitrary, but this seems to work well


class Board:
    def __init__(self):
        self.puzzle: List[List[int]] = [[EMPTY_VALUE]*9 for _ in range(9)]  # Stores the original puzzle
        self.state: List[List[int]] = [[EMPTY_VALUE]*9 for _ in range(9)]   # Stores the current board state
        self.row_masks = [0]*9
        self.col_masks = [0]*9
        self.square_masks = [0]*9

        try:
            self.get_puzzle(PUZZLE_PATH)  # Pulls in the puzzle from a file
        except FileNotFoundError:
            print('Puzzle file not found')
            return

        for row in range(9):  # Copies the puzzle into the board state
            for col in range(9):
                self.set_cell(row, col, self.puzzle[row][col])

        match ALGORITHM:  # Selects the correct solver object
            case 'bt':
                self.solver: BacktrackingSolver = BacktrackingSolver(self)
            case 'fc':
                self.solver: ForwardCheckingSolver = ForwardCheckingSolver(self)
            case 'ac3':
                self.solver: AC3Solver = AC3Solver(self)
            case 'sa':
                self.solver: SimulatedAnnealingSolver = SimulatedAnnealingSolver(self)
            case 'ga':
                self.solver: GeneticSolver = GeneticSolver(self)
            case _:
                print('Unknown algorithm')
                return

        self.solver.solve()  # Runs the solver

        if self.check_solution():
            print("Solved!")
            print(f"Solution decision count: {self.solver.decision_count}")
            print(f"Solution stored at {self.output_puzzle_file()}")
        else:
            print("No solution found.")

    def __str__(self) -> str:
        out: List[str] = []
        for line in self.state:
            out.append(','.join(str(c) if c != EMPTY_VALUE else '?' for c in line))
        return '\n'.join(out) + '\n'

    def pretty_print(self):
        for r in range(9):
            if r % 3 == 0 and r != 0:
                print("-" * (9*2 + 3))  # horizontal separator

            row_str = ""
            for c in range(9):
                if c % 3 == 0 and c != 0:
                    row_str += "| "
                val = self.state[r][c]
                row_str += (str(val) if val != EMPTY_VALUE else ".") + " "
            print(row_str.strip())

    def get_cell(self, row: int, col: int) -> int:
        return self.state[row][col]

    def get_row(self, row: int) -> List[int]:
        return self.state[row]

    def get_col(self, col: int) -> List[int]:
        return [row[col] for row in self.state]

    def set_cell(self, row: int, col: int, value: int):
        self.state[row][col] = value
        if value != EMPTY_VALUE:
            bit = 1 << (value - 1)  # map 1..9 -> bits 0..8
            self.row_masks[row] |= bit
            self.col_masks[col] |= bit
            self.square_masks[(row//3)*3 + (col//3)] |= bit

    def clear_cell(self, row: int, col: int):
        original_value = self.state[row][col]
        if original_value != EMPTY_VALUE:
            bit = 1 << (original_value - 1)
            self.row_masks[row] &= ~bit
            self.col_masks[col] &= ~bit
            self.square_masks[(row//3)*3 + (col//3)] &= ~bit
        self.state[row][col] = EMPTY_VALUE

    def get_row_mask(self, row: int) -> int:
        return self.row_masks[row]

    def get_col_mask(self, col: int) -> int:
        return self.col_masks[col]

    def get_square_mask(self, row: int, col: int) -> int:
        return self.square_masks[(row//3)*3 + (col//3)]

    def get_puzzle(self, filename: str):
        with open(filename) as f:
            for y, line in enumerate(f.readlines()):
                for x, value in enumerate(line.split(',')):
                    value = value.strip()
                    if not value.isnumeric():
                        self.puzzle[y][x] = EMPTY_VALUE
                    else:
                        self.puzzle[y][x] = int(value)

    def output_puzzle_file(self) -> str:
        puzzle_name = ""
        for ch in reversed(PUZZLE_PATH):
            if ch in ('/', '\\'):
                break
            puzzle_name = ch + puzzle_name
        puzzle_name = puzzle_name.split('.')[0]  # Removes txt extension

        os.makedirs('output', exist_ok=True)
        outfile_name = f'output/{GROUP_ID}_{ALGORITHM}_{PUZZLE_TYPE}_{puzzle_name}.txt'
        with open(outfile_name, 'w') as f:
            f.write(str(self))
        return outfile_name

    def check_solution(self) -> bool:
        # 1) No empties
        values: set[int] = set()
        for row in self.state:
            values.update(row)
        if EMPTY_VALUE in values:
            return False

        values = set()
        # 2) Rows and columns contain 9 distinct values (Sudoku assumes 1..9)
        for i in range(9):
            row = set(self.get_row(i))
            col = set(self.get_col(i))
            if len(row) != 9:
                return False
            if len(col) != 9:
                return False

            values = values.union(row)

            # 3) Each 3x3 square distinct
            r0 = (i // 3) * 3
            c0 = (i % 3) * 3
            square = [self.state[r][c] for r in range(r0, r0+3) for c in range(c0, c0+3)]
            if len(set(square)) != 9:
                return False

        # 4) Only legal values in the board (1...9)
        if len(values.intersection(set(range(1, 10)))) != 9:
            return False

        # 5) All original tiles from the puzzle are preserved
        for row in range(9):
            for col in range(9):
                if self.puzzle[row][col] != EMPTY_VALUE and self.state[row][col] != self.puzzle[row][col]:
                    return False

        return True


class BacktrackingSolver:
    def __init__(self, board: Board):
        self.decision_count: int = 0
        self.board: Board = board

    def solve(self) -> bool:
        pos: Optional[Tuple[int, int]] = self.next_empty()
        if pos is None:
            return True

        row, col = pos
        for digit in range(1, 10):
            self.decision_count += 1  # count each attempted assignment
            if self.possible(row, col, digit):
                self.board.set_cell(row, col, digit)
                if self.solve():
                    return True
                self.board.clear_cell(row, col)
        return False

    def possible(self, row: int, col: int, digit: int) -> bool:
        bit = 1 << (digit - 1)
        if self.board.get_row_mask(row) & bit:
            return False
        if self.board.get_col_mask(col) & bit:
            return False
        if self.board.get_square_mask(row, col) & bit:
            return False
        return True

    def next_empty(self) -> Optional[Tuple[int, int]]:
        for row in range(9):
            for col in range(9):
                if self.board.state[row][col] == EMPTY_VALUE:
                    return row, col
        return None


class ForwardCheckingSolver:
    def __init__(self, board: Board):
        self.decision_count: int = 0
        self.board: Board = board

    def solve(self) -> bool:
        pass


class AC3Solver:
    def __init__(self, board: Board):
        self.decision_count: int = 0
        self.board: Board = board

    def solve(self) -> bool:
        pass


class SimulatedAnnealingSolver:
    def __init__(self, board: Board):
        self.decision_count: int = 0
        self.board: Board = board

    def solve(self) -> bool:
        pass


class GeneticSolver:
    def __init__(self, board: Board):
        self.decision_count: int = 0
        self.board: Board = board

    def solve(self) -> bool:
        pass


def main():
    Board().pretty_print()


if __name__ == '__main__':
    main()


Solved!
Solution decision count: 80491
Solution stored at output/Group29_bt_easy_escargot.txt
1 6 2 | 8 5 7 | 4 9 3
5 3 4 | 1 2 9 | 6 7 8
7 8 9 | 6 4 3 | 5 2 1
---------------------
4 7 5 | 3 1 2 | 9 8 6
9 1 3 | 5 8 6 | 7 4 2
6 2 8 | 7 9 4 | 1 3 5
---------------------
3 5 6 | 4 7 8 | 2 1 9
2 4 1 | 9 3 5 | 8 6 7
8 9 7 | 2 6 1 | 3 5 4
